In [1]:
import os
import shutil
from copy import deepcopy
import regex as re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
from importlib import reload
import cleaning as c
import logs
import configs
import files

In [1]:
import spacy

/home/rimov/anaconda3/envs/que_drop/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
spacy.explain('aux')

'auxiliary'

In [2]:
logs.setup_logger('tidy-cleaning',
                  desc='Id duplication checks added to TwitterData. Continuing work on standardizing cleaning section.')

In [13]:
""" Reload changed modules without restarting kernel """
reload(files)

<module 'files' from '/home/rimov/Documents/Code/NLP/lin-que-dropping/src/utils/files.py'>

In [5]:
# Configuration files
gen_conf = configs.read_conf()
conf = configs.read_conf('c')

In [8]:
# Extracted folder to clean
e_path = files.get_saved_data_path('e')

Choose from the available folders, comma-separated (or "a" for all):
0. 26072021-at-1905
1. 07072021-at-1050
2. 2023-02-08-at-23:05:39
3. 26072021-at-2345
4. 2023-02-22-at-18:46:25
5. 07112021-at-2210
6. 26072021-at-1546
7. 2023-02-22-at-18:13:40
Return folder(s): 4


In [9]:
e_name = e_path[0].stem
e_name

'2023-02-22-at-18:46:25'

In [10]:
# Path to the verb conjugations
conjug_es = files.get_verb_conjugations()

### Organize Extracted Files

In [14]:
c_path = files.get_save_path('c', 'twitter')/f'{e_name}'
files.make_dir(files.get_save_path('c', 'twitter'), e_name)

PosixPath('/home/rimov/Documents/Code/NLP/lin-que-dropping/data/cleaned/saved/twitter/es/2023-02-22-at-18:46:25')

if conf['flags']['separate_by_verb']:
    logger.info('Creating separate verb directories')
    for v in conjug_es['verb'].values:
        try:
            utils.make_dir(cleaned_path, v)
        except:
            logger.exception(f'Couldn\'t create directory for "{v}"')

if conf['flags']['separate_by_verb']:
    # MODIFY AS NEEDED
    verb_pat = conf['verb_pat_corpes']
    
    for folder in extracted_folders:
        p = extracted_path/folder
        logger.info(f'Copying files from {p}')
        
        for file in p.iterdir():
            match = regex.search(verb_pat, file.name)
            v = match['keyword'].lower().strip()
            
            try:
                new_name = file.name.replace(' ', '')
                shutil.copyfile(file, cleaned_path/v/new_name)
            except FileNotFoundError as fe:
                logger.exception(fe.args)
                logger.exception(f'Probably no directory for verb: {v}')
                continue

### Cleaning Users

#### TODO: create cleaning module

In [42]:
print(conf['flags']['clean_users'])

True


In [44]:
cleaned = set()

In [45]:
print(cleaned)

set()


In [46]:
if conf['flags']['clean_users']:
    logger.info(f'Marked to clean users.')
    
    cleaned = cleaning.folder_dup_clean(
        cleaned=cleaned, 
        path=dir_cleaned, 
        file_identifier='users', 
        dup_subset='user_id', 
        file_csv_sep='~')

In [47]:
logger.info(f'Cleaned {len(cleaned)} user folders.')

In [48]:
if conf['flags']['clean_users'] is True:
    try:
        conf['flags']['clean_users'] = False
        conf = utils.get_updated_config(conf, 
                                        utils.get_project_root()\
                                          /'cleaning'/'cleaning_config.yml')
    except Exception as e:
        logger.exception(f'Failed to write config file! {e.args}')
    finally:
        logger.info('CONFIG UPDATED: flags.clean_users=False')

### Cleaning Places

In [49]:
os.chdir(dir_cleaned)

print(conf['flags']['clean_places'])

True


In [50]:
cleaned = set()

In [51]:
print(cleaned)

set()


In [52]:
if conf['flags']['clean_places']:
    logger.info(f'Marked to clean places.')
    
    cleaned = cleaning.folder_dup_clean(
        cleaned=cleaned, 
        path=dir_cleaned, 
        file_identifier='places', 
        dup_subset='place_id', 
        file_csv_sep='~')

In [53]:
logger.info(f'Cleaned {len(cleaned)} place folders.')

In [54]:
if conf['flags']['clean_places'] is True:
    try:
        conf['flags']['clean_places'] = False
        conf = utils.get_updated_config(conf, 
                                        utils.get_project_root()\
                                          /'cleaning'/'cleaning_config.yml')
    except Exception as e:
        logger.exception(f'Failed to write config file! {e.args}')
    finally:
        logger.info('CONFIG UPDATED: flags.clean_places=False')

### Cleaning Metadata

In [55]:
os.chdir(dir_cleaned)

print(conf['flags']['clean_meta'])

True


In [56]:
cleaned = set()

In [57]:
print(cleaned)

set()


In [58]:
if conf['flags']['clean_meta']:
    logger.info(f'Marked to clean metadata.')
    
    cleaned = cleaning.folder_dup_clean(
        cleaned=cleaned, 
        path=dir_cleaned, 
        file_identifier='twitterdata', 
        dup_subset='resource_id', 
        file_csv_sep='~')

In [59]:
logger.info(f'Cleaned {len(cleaned)} meta folders.')

In [60]:
if conf['flags']['clean_meta'] is True:
    try:
        conf['flags']['clean_meta'] = False
        conf = utils.get_updated_config(conf, 
                                        utils.get_project_root()\
                                          /'cleaning'/'cleaning_config.yml')
    except Exception as e:
        logger.exception(f'Failed to write config file! {e.args}')
    finally:
        logger.info('CONFIG UPDATED: flags.clean_meta=False')

### Cleaning Tweets

## TODO: 
author_id -> user_id in tweets<br>
format mentions

#### Removing duplicates

In [61]:
os.chdir(dir_cleaned)

print(conf['flags']['clean_dups'])

True


In [62]:
cleaned = set()

In [63]:
print(cleaned)

set()


In [64]:
if conf['flags']['clean_dups']:
    logger.info(f'Marked to clean tweets.')
    
    cleaned = cleaning.folder_dup_clean(
        cleaned=cleaned, 
        path=dir_cleaned, 
        delete_original=True,
        file_identifier='tweets', 
        dup_subset='tweet_id', 
        file_csv_sep='~',
        batch=False,
        batch_size=1000,
        name_scheme='original-tweets')

In [65]:
logger.info(f'Cleaned {len(cleaned)} tweet folders.')

In [66]:
cleaned

(436577, 0)

#### NOTE: 
Above steps only remove duplicates within the partitioned folders, not across them.

## TODO: 
clean 'mentions' column <br>
drop some entries with null values (eg. null author_id)

In [68]:
print(conf['clean']['tweet_combine_col_map'])
print(conf['clean']['drop_cols'])

{'retweet_reply_like_quote': ['public_metrics.retweet_count', 'public_metrics.reply_count', 'public_metrics.like_count', 'public_metrics.quote_count']}
['geo.coordinates.type', 'geo.coordinates.coordinates', 'public_metrics.retweet_count', 'public_metrics.reply_count', 'public_metrics.like_count', 'public_metrics.quote_count']


In [69]:
def clean(df, combine_col_map:dict, drop_cols:list) -> pd.DataFrame:
    
    try:
        # Reformat 'referenced_tweets' column
        df['referenced_tweets'] = df['referenced_tweets'].str.findall(r'\'id\': \'(\d+)')
        
        # Convert dates to datetime object
        df['created_at'] = pd.to_datetime(df['created_at'], format='%Y-%m-%dT%H:%M:%S', errors='coerce')
        
        for col_map in combine_col_map.items():
            # Combine public metrics into 1 column
            df[col_map[0]] = df.apply(cleaning.combine_cols, axis=1, cols=col_map[1])
        
        df.drop(drop_cols, axis=1, inplace=True, errors='ignore')
        
    except ValueError as ve:
        nat = time[time.isnull()]
        
        logging.debug(f'Failed to convert:\n{nat}\n to datetime')
        
    
    return df

In [70]:
cleaned = set()

In [71]:
cleaned

set()

In [72]:
if conf['flags']['clean_table']:
    for folder in Path('.').iterdir():
        if folder in cleaned:
            logging.debug(f'({folder}) is clean')
            continue
            
        logging.info(f'Cleaning ({folder})')
        
        try:
            path = next(Path(folder).rglob('*tweets*.csv'))
            tweets = pd.read_csv(path, sep='~', lineterminator='\n')
            
            df_clean = clean(tweets.copy(), 
                             conf['clean']['tweet_combine_col_map'], 
                             conf['clean']['drop_cols'])
            
            utils.save_csv(path.parent, 
                           df_clean, 
                           f'{folder}-cleaned-original-tweets')
            
        except StopIteration as si:
            logging.exception(f'Folder ({folder}) has no tweets file')
            continue
        
        cleaned.add(folder)

In [73]:
if conf['flags']['clean_table'] is True:
    try:
        conf['flags']['clean_table'] = False
        conf = utils.get_updated_config(conf, 
                                           utils.get_project_root()\
                                             /'cleaning'/'cleaning_config.yml')
    except Exception as e:
        logger.exception(f'Failed to write config file! {e.args}')
    finally:
        logger.info('CONFIG UPDATED: flags.clean_table=False')

### Checking for Query Match

Verify that the query term was used in retrieving tweets

In [74]:
from unidecode import unidecode

In [76]:
os.chdir(utils.get_save_path('c', lang='es', is_test=False)/extracted_save_folders[0])
%ls

acordar/    confesar/    dudar/     mandar/     pedir/       responder/  temer/
adivinar/   confirmar/   entender/  mencionar/  predecir/    rogar/
admitir/    conseguir/   gritar/    mostrar/    prever/      sentir/
afirmar/    considerar/  imaginar/  negar/      prometer/    solicitar/
apostar/    contar/      jurar/     ojala/      reclamar/    suplicar/
asegurar/   demostrar/   lamentar/  ordenar/    recomendar/  suponer/
comprobar/  desear/      lograr/    parecer/    recordar/    suspirar/


In [77]:
indicatives = es_verbs_conjug[['verb', 'indicativo']]
indicatives['indicativo'] = indicatives.loc[:, 'indicativo'].apply(unidecode)

/tmp/ipykernel_230557/3720025898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  indicatives['indicativo'] = indicatives.loc[:, 'indicativo'].apply(unidecode)


In [78]:
def purify_text(tweet):
    purified = tweet.strip().lower()
    pat_punct = r'[^\w\d\s]+'
    pat_space = r'\s+'
    
    purified = regex.sub(pat_punct, '', purified)
    purified = regex.sub(pat_space, ' ', purified)
    return set(purified.split())

In [79]:
def check_conjug(tweet, *conjugs):
    intersect = tweet.intersection(conjugs)
    
    return len(intersect)>0

In [80]:
extr_conf = utils.get_config(utils.get_project_root()/'extraction'/'extraction_config.yml')

In [81]:
cleaned = set(extr_conf['filled_verbs'])
cleaned

{'creer', 'decir', 'esperar', 'pensar', 'querer', 'saber', 'ver'}

In [83]:
matching_tweets = dict()

for folder in Path('.').iterdir():
    if folder in cleaned:
        logging.debug(f'({folder}) is clean')
        continue
    
    
    logging.info(f'Counting properly matched queries in: ({folder})')
    
    match = indicatives['verb']==str(folder)
    conjugs = indicatives[match]['indicativo'].str.split().to_numpy()[0]
    conjugs = set(conjugs)
    
    try:
        path = next(Path(folder).rglob('*cleaned*tweets*.csv'))
        tweets = pd.read_csv(path, sep='~', lineterminator='\n')['text_norm']
        tweets = tweets.apply(purify_text)
        
        has_conjug = tweets.apply(check_conjug, args=(conjugs)).rename('has_conjug')
        matching_tweets[folder] = has_conjug.sum()
        
        print(f'({folder}) yielded: {has_conjug.sum()} out of {tweets.shape[0]}')
        
    except StopIteration as si:
        logging.exception(f'Folder ({folder}) has no tweets file')
        continue

(confirmar) yielded: 4751 out of 4801
(dudar) yielded: 20420 out of 20723
(suplicar) yielded: 291 out of 295
(parecer) yielded: 47629 out of 48102
(solicitar) yielded: 1703 out of 1708
(afirmar) yielded: 1194 out of 1218
(temer) yielded: 1880 out of 1915
(mencionar) yielded: 2641 out of 2660
(suspirar) yielded: 356 out of 360
(contar) yielded: 49848 out of 50293
(prometer) yielded: 3907 out of 3956
(jurar) yielded: 6272 out of 6357
(admitir) yielded: 917 out of 926
(rogar) yielded: 955 out of 987
(asegurar) yielded: 2754 out of 2778
(comprobar) yielded: 1109 out of 1125
(mostrar) yielded: 6907 out of 6956
(conseguir) yielded: 6665 out of 6721
(recomendar) yielded: 4736 out of 4772
(lamentar) yielded: 2133 out of 2165
(apostar) yielded: 2889 out of 2923
(predecir) yielded: 152 out of 154
(desear) yielded: 11285 out of 11389
(pedir) yielded: 26506 out of 26685
(acordar) yielded: 28703 out of 28966
(sentir) yielded: 50036 out of 50282
(adivinar) yielded: 772 out of 790
(suponer) yielded: 

### Merging whole query

## TODO:
Generalize above and below duplication finding steps and add to cleaning module

In [84]:
es_save_dir = utils.get_save_path('c', is_test=False, lang='es')
os.chdir(es_save_dir)

utils.make_dir(es_save_dir/f'{extracted_save_folders[0]}-combined')
os.chdir(es_save_dir/f'{extracted_save_folders[0]}-combined')

In [85]:
tweet_paths = Path(dir_cleaned).rglob('*cleaned*tweets*.csv')
user_paths = Path(dir_cleaned).rglob('*users*.csv')
places_paths = Path(dir_cleaned).rglob('*places*.csv')
meta_paths = Path(dir_cleaned).rglob('*twitterdata*.csv')

In [86]:
users = pd.concat([pd.read_csv(p, sep='~') for p in user_paths]).reset_index(drop=True)
places = pd.concat([pd.read_csv(p, sep='~') for p in places_paths]).reset_index(drop=True)
meta = pd.concat([pd.read_csv(p, sep='~') for p in meta_paths]).reset_index(drop=True)

In [87]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 671167 entries, 0 to 671166
Data columns (total 10 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   name                            671124 non-null  object
 1   username                        671167 non-null  object
 2   user_id                         671167 non-null  int64 
 3   created_at                      671167 non-null  object
 4   user_location                   493598 non-null  object
 5   public_metrics.followers_count  671167 non-null  int64 
 6   public_metrics.following_count  671167 non-null  int64 
 7   public_metrics.tweet_count      671167 non-null  int64 
 8   public_metrics.listed_count     671167 non-null  int64 
 9   withheld.country_codes          3 non-null       object
dtypes: int64(5), object(5)
memory usage: 51.2+ MB


In [88]:
places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64790 entries, 0 to 64789
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   country             64789 non-null  object
 1   location_full_name  64790 non-null  object
 2   place_id            64790 non-null  object
dtypes: object(3)
memory usage: 1.5+ MB


In [89]:
meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   value          105 non-null    object
 1   detail         105 non-null    object
 2   title          105 non-null    object
 3   resource_type  105 non-null    object
 4   parameter      105 non-null    object
 5   resource_id    105 non-null    object
 6   type           105 non-null    object
dtypes: object(7)
memory usage: 5.9+ KB


In [90]:
tweets = pd.concat([pd.read_csv(p, sep='~', lineterminator='\n') for p in tweet_paths]).reset_index(drop=True)
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 436577 entries, 0 to 436576
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   created_at                436577 non-null  object
 1   text_orig                 436577 non-null  object
 2   author_id                 436577 non-null  int64 
 3   lang                      436577 non-null  object
 4   tweet_id                  436577 non-null  int64 
 5   tweet_place_id            436573 non-null  object
 6   referenced_tweets         271143 non-null  object
 7   mentions                  278317 non-null  object
 8   text_norm                 436577 non-null  object
 9   retweet_reply_like_quote  436577 non-null  object
dtypes: int64(2), object(8)
memory usage: 33.3+ MB


In [91]:
user_dups = users.duplicated(subset='user_id')
place_dups = places.duplicated(subset='place_id')
meta_dups = meta.duplicated(subset='resource_id')

In [92]:
tweet_dups = tweets.duplicated(subset='tweet_id')

In [93]:
user_dups_total = user_dups.sum()
place_dups_total = place_dups.sum()
meta_dups_total = meta_dups.sum()

In [94]:
tweet_dups_total = tweet_dups.sum()

In [95]:
print(tweet_dups_total, user_dups_total, place_dups_total, meta_dups_total)

28213 373131 53732 24


In [96]:
users.drop(users[user_dups].index, inplace=True)
places.drop(places[place_dups].index, inplace=True)
meta.drop(meta[meta_dups].index, inplace=True)

In [97]:
tweets.drop(tweets[tweet_dups].index, inplace=True)

In [98]:
print(tweets.shape[0], users.shape[0], places.shape[0], meta.shape[0])

408364 298036 11058 81


In [99]:
utils.save_csv(Path('.'), tweets, 'tweets')
utils.save_csv(Path('.'), users, 'users')
utils.save_csv(Path('.'), places, 'places')
utils.save_csv(Path('.'), meta, 'twitterdata')

#### Mark flagged accounts

In [100]:
flagged_users = set(meta['value'])

flagged = users['username'].isin(flagged_users)
flagged.sum()

0

In [101]:
users['flagged'] = flagged
users.head()

,name,username,user_id,created_at,user_location,public_metrics.followers_count,public_metrics.following_count,public_metrics.tweet_count,public_metrics.listed_count,withheld.country_codes,flagged
0,Reztre🎄,Reztre,141323312,2010-05-07T19:44:51.000Z,"Pereira, Risaralda",1016,687,27049,13,NaN,False
1,Carmen,carmen_flurry,49454158,2009-06-21T22:42:22.000Z,NaN,2533,3425,158751,79,NaN,False
2,Karla Peregrina,karlapo,26833188,2009-03-26T19:54:35.000Z,"Cancún, Q. Roo / CDMX",818,1760,6112,8,NaN,False
3,Gabriela Damián Miravete,gabrielintica,146562887,2010-05-21T19:15:47.000Z,Falso Bosque,9164,3530,36661,105,NaN,False
4,Rebeca Jiménez Calero,rebecajc,157111888,2010-06-18T22:16:20.000Z,"México, D.F.",1845,1610,36772,47,NaN,False


In [102]:
utils.save_csv(Path('.'), users, 'users')

### Checking against previous pulls

In [105]:
es_save_folder = ['combined']
es_save_path = utils.get_save_path('c', lang='es')/'20210726'/es_save_folder[0]

In [106]:
prev_tweets = pd.read_csv(es_save_path/'tweets.csv', sep='~')
prev_users = pd.read_csv(es_save_path/'users.csv', sep='~')
prev_places = pd.read_csv(es_save_path/'places.csv', sep='~')
prev_meta = pd.read_csv(es_save_path/'twitterdata.csv', sep='~')

/home/rimov/anaconda3/envs/nlp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [107]:
prev_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560093 entries, 0 to 560092
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   created_at                560089 non-null  object 
 1   lang                      560091 non-null  object 
 2   author_id                 560087 non-null  float64
 3   referenced_tweets         353987 non-null  object 
 4   tweet_id                  560088 non-null  float64
 5   text_orig                 560090 non-null  object 
 6   mentions                  366625 non-null  object 
 7   place_id                  560067 non-null  object 
 8   text_norm                 560090 non-null  object 
 9   retweet_reply_like_quote  560088 non-null  object 
dtypes: float64(2), object(8)
memory usage: 42.7+ MB


In [112]:
# Rename non-matching columns
prev_tweets.rename(columns={'place_id': 'tweet_place_id'}, inplace=True)

In [108]:
prev_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327265 entries, 0 to 327264
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   name                            327234 non-null  object
 1   author_id                       327265 non-null  int64 
 2   username                        327265 non-null  object
 3   created_at                      327265 non-null  object
 4   user_location                   230091 non-null  object
 5   public_metrics.followers_count  327265 non-null  int64 
 6   public_metrics.following_count  327265 non-null  int64 
 7   public_metrics.tweet_count      327265 non-null  int64 
 8   public_metrics.listed_count     327265 non-null  int64 
 9   withheld.country_codes          3 non-null       object
 10  flagged                         327265 non-null  bool  
dtypes: bool(1), int64(5), object(5)
memory usage: 25.3+ MB


In [111]:
# Rename non-matching columns
prev_users.rename(columns={'author_id': 'user_id'}, inplace=True)

In [109]:
prev_places.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14381 entries, 0 to 14380
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   country   14377 non-null  object
 1   location  14381 non-null  object
 2   place_id  14381 non-null  object
dtypes: object(3)
memory usage: 337.2+ KB


In [113]:
# Rename non-matching columns
prev_places.rename(columns={'location': 'location_full_name'}, inplace=True)

In [110]:
prev_meta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   parameter      1035 non-null   object
 1   resource_id    1035 non-null   object
 2   value          1035 non-null   object
 3   detail         1035 non-null   object
 4   title          1035 non-null   object
 5   resource_type  1035 non-null   object
 6   type           1035 non-null   object
dtypes: object(7)
memory usage: 56.7+ KB


In [116]:
merged_tweets = pd.concat([tweets, prev_tweets], axis=0).reset_index(drop=True)
merged_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968457 entries, 0 to 968456
Data columns (total 10 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   created_at                968453 non-null  object 
 1   text_orig                 968454 non-null  object 
 2   author_id                 968451 non-null  float64
 3   lang                      968455 non-null  object 
 4   tweet_id                  968452 non-null  float64
 5   tweet_place_id            968427 non-null  object 
 6   referenced_tweets         607203 non-null  object 
 7   mentions                  626598 non-null  object 
 8   text_norm                 968454 non-null  object 
 9   retweet_reply_like_quote  968452 non-null  object 
dtypes: float64(2), object(8)
memory usage: 73.9+ MB


In [117]:
merged_users = pd.concat([users, prev_users], axis=0).reset_index(drop=True)
merged_places = pd.concat([places, prev_places], axis=0).reset_index(drop=True)
merged_meta = pd.concat([meta, prev_meta], axis=0).reset_index(drop=True)

In [118]:
user_dups = merged_users.duplicated(subset='user_id')
place_dups = merged_places.duplicated(subset='place_id')
meta_dups = merged_meta.duplicated(subset='resource_id')

In [119]:
tweet_dups = merged_tweets.duplicated(subset='tweet_id')

In [120]:
user_dups_total = user_dups.sum()
place_dups_total = place_dups.sum()
meta_dups_total = meta_dups.sum()

In [121]:
tweet_dups_total = tweet_dups.sum()

In [122]:
print(tweet_dups_total, user_dups_total, place_dups_total, meta_dups_total)

88356 96125 7147 12


In [123]:
merged_users.drop(merged_users[user_dups].index, inplace=True)
merged_places.drop(merged_places[place_dups].index, inplace=True)
merged_meta.drop(merged_meta[meta_dups].index, inplace=True)

In [124]:
merged_tweets.drop(merged_tweets[tweet_dups].index, inplace=True)

In [126]:
print(merged_tweets.shape[0], merged_users.shape[0], merged_places.shape[0], merged_meta.shape[0])

880101 529176 18292 1104


In [99]:
utils.save_csv(Path('.'), tweets, 'tweets')
utils.save_csv(Path('.'), users, 'users')
utils.save_csv(Path('.'), places, 'places')
utils.save_csv(Path('.'), meta, 'twitterdata')

#### Separate and save

In [135]:
new_tweets = ~merged_tweets['tweet_id'].isin(prev_tweets['tweet_id'])
new_users = ~merged_users['user_id'].isin(prev_users['user_id'])
new_places = ~merged_places['place_id'].isin(prev_places['place_id'])
new_meta = ~merged_meta['resource_id'].isin(prev_meta['resource_id'])
print(f'tweets: {new_tweets.sum()}, users: {new_users.sum()}, places: {new_places.sum()}, meta: {new_meta.sum()}')

tweets: 408362, users: 201911, places: 3911, meta: 69


In [136]:
tweets = merged_tweets[new_tweets].copy()
users = merged_users[new_users].copy()
places = merged_places[new_places].copy()
meta = merged_meta[new_meta].copy()

In [141]:
os.chdir(es_save_dir)
os.chdir(es_save_dir/f'{extracted_save_folders[0]}-combined')

In [142]:
utils.save_csv(Path('.'), tweets, 'tweets')
utils.save_csv(Path('.'), users, 'users')
utils.save_csv(Path('.'), places, 'places')
utils.save_csv(Path('.'), meta, 'twitterdata')